In [10]:
from pdf_rag.utils import rag_chain, process_pdfs_from_folder
import gradio as gr
import os
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings


### Creating Gradio interface

In [11]:
def ask_question(folder_path, question):
    if not folder_path or not os.path.exists(folder_path):
        return "Invalid folder path. Please enter a valid directory."

    vectorstore, retriever = process_pdfs_from_folder(folder_path)

    if retriever is None:
        return "No valid PDFs found in the folder."

    result = rag_chain(question, retriever)
    return result

interface = gr.Interface(
    fn=ask_question,
    inputs=[
        gr.FileExplorer(root_dir="./data", label="PDF Folder Path"),
        gr.Textbox(label="Ask a question"),
    ],
    outputs="text",
    title="Ask questions about PDFs in a folder",
    description="Enter the path to a folder containing PDFs and ask questions about them.",
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
